In [1]:
import pandas as pd 
import time 
import datetime 
import MetaTrader5 as mt5
import  logging
import os
logging.basicConfig(filename='bot.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
def create_trade_history_df(symbol):
    cols = [    
                "trade_date_id",
                "create_order_date",
                "buy_ticket",
                "sell_ticket",
                "account_balance",
                "account_equity",
                "account_margin",
                "current_market_price",
                "buy_volume",
                "sell_volume",
                "buy_at",
                "sell_at",
                "first_buy_at",
                "first_sell_at",
                "buy_status",
                "sell_status",
                "fib_number",
                "next_fib_number",
                "next_volume",
                "next_buy_at",
                "next_sell_at",
                "market_direction",
                "close_all_order_type",
                "close_all_when_market_reverse_price_at",
                "is_first_order_of_sequence",
            ]
    df_file_name = f"{symbol}_trade_history_df.csv"
    
    # Check if the file already exists
    if os.path.exists(df_file_name):
        logging.info(f"File {df_file_name} already exists. Skipping creation.")
        return

    trade_history_df = pd.DataFrame(columns=cols)
    trade_history_df.to_csv(df_file_name, index=None)
    logging.info(f"Data frame has been created. File name: {df_file_name}")


In [3]:
def make_order(symbol, volume, order_type):
        tick = mt5.symbol_info_tick(symbol)
        if not tick:
            logging.info("Failed to fetch tick for", symbol)
            return None
        
        order_dict = {'buy': 0, 'sell': 1}
        price_dict = {'buy': tick.ask, 'sell': tick.bid}

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_dict[order_type],
            "price": price_dict[order_type],
            "deviation": 20,
            "magic": 100,
            "comment": "python market order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        order_result = mt5.order_send(request)
        if not order_result or order_result.retcode != mt5.TRADE_RETCODE_DONE:
            logging.info("Failed to send order:", order_result.comment if order_result else "Unknown error")
            return None
        logging.info(f"order_result: {order_result}")
        return order_result

In [4]:
def close_last_order(symbol,type,ticket,volume):
        tick = mt5.symbol_info_tick(symbol)
        trade_history_df = pd.read_csv(f"{symbol}_trade_history_df.csv")
        
        if not tick:
            logging.info(f"Failed to fetch tick for {symbol}")
            return None
        if type == "buy":
              last_order_price = float(trade_history_df["buy_at"].iloc[-1])
              order_type = mt5.ORDER_TYPE_SELL           
        elif type == "sell":
              last_order_price = float(trade_history_df["sell_at"].iloc[-1])
              order_type = mt5.ORDER_TYPE_BUY
              

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "position": int(ticket),
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": last_order_price,
            "deviation": 20,
            "magic": 100,
            "comment": "python script close",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        
        result = mt5.order_send(request)
        if result and result.retcode == mt5.TRADE_RETCODE_DONE:
            # Update the status in the DataFrame
            if type == "buy":
                trade_history_df["buy_status"].iloc[-1] = "close"
            elif type == "sell":
                trade_history_df["sell_status"].iloc[-1] = "close"

            # Write the updated DataFrame back to the CSV file
            trade_history_df.to_csv(f"{symbol}_trade_history_df.csv", index=False)
        else:
            logging.info("Failed to send order. Return code:", result.retcode if result else "No result")
            logging.info("Error:", mt5.last_error())  # This will print the last error from MT5

        return result

In [5]:
def has_open_orders(symbol):
    trade_history_df = pd.read_csv(f"{symbol}_trade_history_df.csv")
    if trade_history_df.empty:
        return False
    
    last_trade_buy_status = trade_history_df["buy_status"].iloc[-1]
    last_trade_sell_status = trade_history_df["sell_status"].iloc[-1]
    logging.info(f"last_trade_buy_status : {last_trade_buy_status}")
    logging.info(f"last_trade_sell_status : {last_trade_sell_status}")
    
    if last_trade_buy_status == "open" and last_trade_sell_status == "open":
        return True
    elif last_trade_buy_status == "close" and last_trade_sell_status == "close":
        return False
    else:
        # Handle other cases here, or raise an exception if these cases shouldn't occur
        logging.info("Unexpected trade status combination.")
        return False  # or raise an exception


In [6]:
def get_current_market_price(symbol):
        tick = mt5.symbol_info_tick(symbol)
        if not tick:
            logging.info(f"Failed to fetch tick for {symbol}")
            return None
        return tick.ask  # or tick.bid based on the requirement

In [7]:
def make_first_order(symbol,volume):
    account_info = mt5.account_info()
    first_fib_number = 1
    next_fib_number = 2
    
    logging.info(f"Account Balance: {account_info.balance}")
    logging.info(f"Account Equity: {account_info.equity}")
    logging.info(f"Account Margin: {account_info.margin}")
    logging.info(f"Account Free Margin: {account_info.margin_free}")
    logging.info(f"fib number :{first_fib_number}")

    symbol_properties = mt5.symbol_info(symbol)
    if symbol_properties:
        logging.info(f"Symbol Margin Required for 1 lot: {symbol_properties.margin_hedged}")
    
    create_trade_history_df(symbol)

    current_market_price = get_current_market_price(symbol)

    make_buy_order = make_order(symbol,volume,"buy")
    make_sell_order = make_order(symbol,volume,"sell")

    next_buy_volume = volume * float(make_buy_order.price)
    next_sell_volume = volume * float(make_sell_order.price)
    
    # Create a new row with the values you want to append
    first_trade_row = {
        "trade_date_id": pd.Timestamp.now().strftime("%d%m%y%H%M%S"),
        "buy_ticket":make_buy_order.order,
        "sell_ticket":make_sell_order.order,
        "create_order_date": pd.Timestamp.now(),  # Current timestamp
        "account_balance": account_info.balance,
        "account_equity": account_info.equity,
        "account_margin": account_info.margin,
        "current_market_price" : current_market_price,
        "buy_volume": make_buy_order.volume,
        "sell_volume": make_sell_order.volume,
        "buy_at": make_buy_order.price,
        "sell_at": make_sell_order.price,
        "first_buy_at": make_buy_order.price,
        "first_sell_at": make_sell_order.price,
        "buy_status": "open",
        "sell_status": "open",
        "fib_number": first_fib_number,
        "next_fib_number": next_fib_number,
        "next_volume": next_fib_number * volume,
        "next_buy_at": make_buy_order.price + next_buy_volume,
        "next_sell_at": make_sell_order.price - next_sell_volume,
        "market_direction": None,
        "close_all_order_type": None,
        "close_all_when_market_reverse_price_at": None,
        "is_first_order_of_sequence": True  # Assuming this is the first order of the sequence
    }
    trade_history_df = pd.read_csv(f"{symbol}_trade_history_df.csv")
    # Append the new row to the DataFrame
    trade_history_df = pd.concat([trade_history_df, pd.DataFrame([first_trade_row])], ignore_index=True)

    # Save the updated DataFrame back to the CSV
    trade_history_df.to_csv(f"{symbol}_trade_history_df.csv", index=False)

    logging.info(f"Data frame has been updated. File name: {symbol}_trade_history_df.csv")

In [8]:
def make_next_buy_sell_order(symbol,volume,market_direction):
    # Get dataframe
    trade_history_df = pd.read_csv(f"{symbol}_trade_history_df.csv")
    
    last_fib_number = float(trade_history_df["fib_number"].iloc[-1])
    fib_number = float(trade_history_df["next_fib_number"].iloc[-1])
    next_fib_number = last_fib_number + fib_number
    first_buy_at = trade_history_df["first_buy_at"].iloc[-1]
    first_sell_at = trade_history_df["first_sell_at"].iloc[-1]
    
    buy_volume = fib_number * volume
    sell_volume = fib_number * volume

    # get account info
    account_info = mt5.account_info()
    logging.info(f"Account Balance: {account_info.balance}")
    logging.info(f"Account Equity: {account_info.equity}")
    logging.info(f"Account Margin: {account_info.margin}")
    logging.info(f"Account Free Margin: {account_info.margin_free}")

    
    # Get current market price
    current_market_price = get_current_market_price(symbol)


    make_buy_order = make_order(symbol,buy_volume,"sell")
    make_sell_order = make_order(symbol,sell_volume,"buy")

    next_buy_volume = buy_volume * float(make_buy_order.price)
    next_sell_volume = sell_volume * float(make_sell_order.price)

    if market_direction == "buy":
        close_all_order_type = "sell"
        close_all_when_market_reverse_price_at = make_buy_order.price - 0.30 * (make_buy_order.price - first_buy_at)

    elif market_direction == "sell":
        close_all_order_type = "buy"
        close_all_when_market_reverse_price_at = make_sell_order.price + 0.30 * (first_sell_at - make_sell_order.price)

    
    # Create a new row with the values you want to append
    next_trade_row = {
        "trade_date_id": pd.Timestamp.now().strftime("%d%m%y%H%M%S"),
        "buy_ticket":make_buy_order.order,
        "sell_ticket":make_sell_order.order,
        "create_order_date": pd.Timestamp.now(),  # Current timestamp
        "account_balance": account_info.balance,
        "account_equity": account_info.equity,
        "account_margin": account_info.margin,
        "current_market_price" : current_market_price,
        "buy_volume": make_buy_order.volume,
        "sell_volume": make_sell_order.volume,
        "buy_at": make_buy_order.price,
        "sell_at": make_sell_order.price,
        "first_buy_at": first_buy_at,
        "first_sell_at": first_sell_at,
        "buy_status": "open",
        "sell_status": "open",
        "fib_number": int(fib_number),
        "next_fib_number": next_fib_number,
        "next_volume": next_fib_number * volume,
        "next_buy_at": make_buy_order.price + next_buy_volume,
        "next_sell_at": make_sell_order.price - next_sell_volume,
        "market_direction": market_direction,
        "close_all_order_type": close_all_order_type,
        "close_all_when_market_reverse_price_at": close_all_when_market_reverse_price_at,
        "is_first_order_of_sequence": False  # Assuming this is the first order of the sequence
    }
    
    # Append the new row to the DataFrame
    trade_history_df = pd.concat([trade_history_df, pd.DataFrame([next_trade_row])], ignore_index=True)

    # Save the updated DataFrame back to the CSV
    trade_history_df.to_csv(f"{symbol}_trade_history_df.csv", index=False)

    logging.info(f"Data frame has been updated. File name: {symbol}_trade_history_df.csv")

In [9]:
def close_position(position):
        tick = mt5.symbol_info_tick(position.symbol)
        if not tick:
            print(f"Failed to fetch tick for {position.symbol}")
            return None

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "position": position.ticket,
            "symbol": position.symbol,
            "volume": position.volume,
            "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
            "price": tick.ask if position.type == 1 else tick.bid,
            "deviation": 20,
            "magic": 100,
            "comment": "python script close",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)
        return result

In [10]:
def close_all_when_market_reverse_price_at_30pct(symbol):
        position = mt5.positions_get(symbol=symbol)
        for position in position:
            close_position(position)

In [11]:
def run(symbol,volume):
    logging.info(f"PROGRAM START")
    create_trade_history_df(symbol)
    is_has_open_orders = has_open_orders(symbol)
    logging.info(is_has_open_orders)
    
    if is_has_open_orders == False:
        logging.info(f"Make first trade")
        make_first_order(symbol,volume)
    
    elif is_has_open_orders == True:
        logging.info(f"Already has opened order")

    while True:
        trade_history_df = pd.read_csv(f"{symbol}_trade_history_df.csv")
        current_market_price = get_current_market_price(symbol)
        next_buy_at = trade_history_df["next_buy_at"].iloc[-1]
        next_sell_at = trade_history_df["next_sell_at"].iloc[-1]
        trade_current_direction = trade_history_df["market_direction"].iloc[-1]
        close_all_when_market_reverse_price_at = float(trade_history_df["close_all_when_market_reverse_price_at"].iloc[-1])
        logging.info(f"current_market_price: {current_market_price}")
        logging.info(f"next_buy_at: {next_buy_at},next_sell_at: {next_sell_at}")
        
        # DCA for Buy direction
        if current_market_price >= next_buy_at: 
            logging.info("current_market_price >= next_buy_at")
            logging.info(f"Close current buy order")
            buy_ticket = trade_history_df["buy_ticket"].iloc[-1]
            buy_volume = trade_history_df["buy_volume"].iloc[-1]
            close_last_order(symbol,"buy",buy_ticket,buy_volume)
            make_next_buy_sell_order(symbol,volume,"buy")
        
        # DCA for Sell direction
        elif current_market_price <= next_sell_at: 
            logging.info("current_market_price <= next_sell_at")
            logging.info(f"Close current sell order")
            sell_ticket = trade_history_df["sell_ticket"].iloc[-1]
            sell_volume = trade_history_df["sell_volume"].iloc[-1]
            close_last_order(symbol,"sell",sell_ticket,sell_volume)
            make_next_buy_sell_order(symbol,volume,"sell")

        elif current_market_price >= next_sell_at and current_market_price <= next_buy_at:
            logging.info(f"Keep holding")

        elif trade_current_direction == "buy" and close_all_when_market_reverse_price_at is not None and current_market_price < close_all_when_market_reverse_price_at:
            print(f"Market reverse from buy direction")
            close_all_when_market_reverse_price_at_30pct(symbol)
        elif trade_current_direction == "sell" and close_all_when_market_reverse_price_at is not None and current_market_price > close_all_when_market_reverse_price_at:
            print(f"Market reverse from buy direction") 
            close_all_when_market_reverse_price_at_30pct(symbol)
        

        logging.info(f"all position: {mt5.positions_get(symbol=symbol)}")
        time.sleep(30)
    

In [12]:
mt5.initialize(login=51446835, server="ICMarketsSC-Demo", password="qwfgKZdZ")
run("BNBUSD",volume = 0.01)

C:\Users\Pham Quoc Bao\AppData\Local\Temp\ipykernel_3632\3069668657.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_history_df = pd.concat([trade_history_df, pd.DataFrame([first_trade_row])], ignore_index=True)
